In [3]:
search = "Restaurants in new york, usa"
urls = f"https://www.google.com/maps/search/{search.replace(' ', '+')}"
print(urls)

https://www.google.com/maps/search/Restaurants+in+new+york,+usa


In [2]:
search = "Crown Shy"
url = f"https://www.google.com/maps/search/{search.replace(' ', '+')}"
url

'https://www.google.com/maps/search/Crown+Shy'

In [4]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import warnings
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
warnings.simplefilter(action='ignore', category=FutureWarning)

In [120]:
# Make browser open in background
options = webdriver.ChromeOptions()
# options.add_argument('headless')

In [121]:
s = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=s, options=options)

print("Getting business info", urls)
driver.get(urls)
c = driver.page_source
driver.close

In [3]:
def load_companies(url):
    options = webdriver.ChromeOptions()
    s = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=s, options=options)
    print("Getting business info", url)
    driver.get(url)
    
    # Use WebDriverWait to wait for the panel to be present on the page
    panel_xpath = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]'
    wait = WebDriverWait(driver, 10)
    scrollable_div = wait.until(EC.presence_of_element_located((By.XPATH, panel_xpath)))

    # scrolling
    flag = True
    i = 0
    columns = ["Name", "Rating", "Reviews", "Address", "Category", "Time"]
    df = pd.DataFrame(columns=columns)
    
    while flag:
        print(f"Scrolling to page {i + 2}")
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        time.sleep(5)

        if "You've reached the end of the list." in driver.page_source:
            flag = False
        
        df = get_business_info(df, driver.page_source)
        i += 1
        # print(df)
        
    driver.close()
    return df

In [157]:
def load_companies(url, driver):
    options = webdriver.ChromeOptions()
    print("Getting business info", url)
    driver.get(url)
    time.sleep(5)
    panel_xpath = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]'
    scrollable_div = driver.find_element(By.XPATH, panel_xpath)
    # scrolling
    flag = True
    i = 0
    columns = ["Name", "Rating", "Reviews", "Address", "Category", "Time"]
    df = pd.DataFrame(columns=columns)
    while flag:
        print(f"Scrolling to page {i + 2}")
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        time.sleep(5)

        if "You've reached the end of the list." in driver.page_source:
            flag = False
        
        df = get_business_info(df, driver.page_source)
        i += 1
        # print(df)
    driver.close()
    return df

In [4]:
def get_name(content):
    try:
        name = content.find("div", {"class": "qBF1Pd"}).text
    except:
        name = None
    return name

In [5]:
def get_ratings(content):
    try:
        rating = content.find_all("div", {"class": "W4Efsd"})[0].text
        rating = rating.split("(")[0].strip()
    except:
        rating = None
    return rating

In [6]:
def get_reviews(content):
    try:
        review = content.find_all("div", {"class": "W4Efsd"})[0].text
        review = review.split("(")[1].split(")")[0].strip()
    except:
        review = None
    return review

In [7]:
def get_address(content):
    try:
        address = content.find_all("div", {"class": "W4Efsd"})[2].text.split("·")[1]
    except:
        address = None
    return address

In [8]:
def get_category(content):
    try:
        category = content.find_all("div", {"class": "W4Efsd"})[2].text.split("·")[0]
    except:
        category = None
    return category

In [9]:
def get_time(content):
    try:
        time = content.find_all("div", {"class": "W4Efsd"})[4].text.split("⋅")[1]
    except:
        time = None
    return time

In [10]:
def get_business_info(df, content):
    soup = BeautifulSoup(content, "html.parser")
    for i in soup.find_all("div", {"class": "THOPZb"}):
        name = get_name(i)
        rating = get_ratings(i)
        review = get_reviews(i)
        address = get_address(i)
        category = get_category(i)
        time = get_time(i)
        dct = {
            "Name" : name,
            "Rating": rating,
            "Reviews": review,
            "Address": address,
            "Category": category,
            "Time": time
        }
        is_duplicate = dct["Name"] in df["Name"].values
        
        # Append the dictionary DataFrame only if it doesn't already exist in the "name" column
        if not is_duplicate:
            df = df.append(dct, ignore_index=True)
    
    print(df.tail(1))
    print("----------")
    return df

In [11]:
df = load_companies(urls)

Getting business info https://www.google.com/maps/search/Restaurants+in+new+york,+usa
Scrolling to page 2
         Name Rating Reviews                       Address   Category  \
10  Crown Shy    4.6   1,019   70 Pine Street Ground Floor  American    

              Time  
10   Opens 5:30 pm  
----------
Scrolling to page 3
                Name Rating Reviews         Address   Category            Time
15  Restaurant Above    4.3      37   234 W 42nd St  American    Opens 6:30 am
----------
Scrolling to page 4
         Name Rating Reviews      Address Category             Time
20  Kuu Ramen    4.4   1,493   20 John St   Ramen    Opens 11:30 am
----------
Scrolling to page 5
               Name Rating Reviews        Address       Category  \
21  Gramercy Tavern    4.6   3,386   42 E 20th St  New American    

               Time  
21   Opens 11:30 am  
----------
Scrolling to page 6
          Name Rating Reviews       Address       Category             Time
25  The Modern    4.6   1,874 

In [12]:
df

,Name,Rating,Reviews,Address,Category,Time
0,Victory Sweet Shop/Victory Garden Cafe,4.5,342,21-69 Steinway St,Restaurant,Opens 8 am
1,Berimbau Brazilian Kitchen,4.5,958,43 Carmine St,Brazilian,Opens 11:30 am
2,Per Se,4.5,"1,525",10 Columbus Cir,French,Opens 4:30 pm
3,Keens Steakhouse,4.6,"7,028",72 W 36 St,Steak,Opens 11:45 am
4,Marc Forgione,4.6,580,134 Reade St,American,Opens 5 pm
...,...,...,...,...,...,...
108,Nocello,4.6,304,257 W 55th St,Italian,Opens 4 PM
109,Latin American,4.2,237,29 W 26th St # 1,Dominican,Opens 7 AM
110,Buddha-Bar Restaurant New York,4.0,481,62 Thomas St,Asian Fusion,None
111,Tiny's & The Bar Upstairs,4.4,822,135 W Broadway,American,Opens 5 PM


In [13]:
df.to_csv("new_data.csv")

In [15]:
def url_generator(name, address, category = ''):
    search = name+' '+address+' '+category
    url = f"https://www.google.com/maps/search/{search.replace(' ', '+')}"
    print(url)
    return url

In [21]:
def xtractor(df):
    print("here")
    print(df.head())
    info = []
    df = df.reset_index(drop = True)
    for i in range(len(df)):
        url = url_generator(df["Name"][i], df["Address"][i])
        information = information_extractor(url)
        print(information)
        if information:
            if df["Name"][i] not in information[0]["Name"]:
                url = url_generator(df["Name"][i], df["Address"][i], df['Category'][i])
                information = information_extractor(url)
                print(information)
        else:
            url = url_generator(df["Name"][i], df["Address"][i], df['Category'][i])
            information = information_extractor(url)
            print(information)
            
        info.extend(information)
    
    return info

In [18]:
def phone_numbr(content):
    info = []
    pattern = re.compile(r'[a-zA-Z0-9]+\.+[a-zA-Z]+')
    try:
        record = {}
        record["Name"] = content.find("h1", {"class", "DUwDvf lfPIob"}).text
        for i in content.find_all("div", {"class","RcCsl fVHpi w4vB1d NOE9ve M0S7ae AG25L"}):
            if "Address" not in record.keys():
                record["Address"] = i.text
            if '+1' in i.text:
                record["Phone"] = i.text
            if pattern.search(i.text):
                record["Website"] = pattern.search(i.text).group(0)
        # print(record)
        for i in content.find_all("span"):
            # print(i.text.lower())
            if "opens" in i.text.lower():
                record["Time"] = i.text.split("⋅")[1].replace("\u202f", " ")
        info.append(record)
        # print(record)
    except:
        info = []
    return info

In [19]:
def phone_number(content):
    info = []
    pattern = re.compile(r'[a-zA-Z0-9]+\.+[a-zA-Z]+')
    try:
        record = {}
        record["Name"] = content.find("h1", {"class", "DUwDvf lfPIob"}).text
        record["Time"] = []
        for i in content.find_all("div", {"class","fontBodyMedium"}):
            if "Address" not in record.keys():
                record["Address"] = i.text
            if '+1' in i.text:
                record["Phone"] = i.text
            if pattern.search(i.text):
                record["Website"] = pattern.search(i.text).group(0)
            else:
                record["Time"].append(i.text)
        info.append(record)
        # print(record)
    except:
        info = []
    return info

In [20]:
def information_extractor(url):
    s = Service(ChromeDriverManager().install())
    # options.add_argument("--headless")
    driver = webdriver.Chrome(service=s, options=options)
    driver.get(url)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, "html")
    # address = new_address(soup)
    ph = phone_numbr(soup)
    # print(address)
    # print(ph)
    return ph

In [101]:
x(df[2:6])

https://www.google.com/maps/search/Per+Se++10+Columbus+Cir+
{'Name': 'Per Se', 'Address': '10 Columbus Cir, New York, NY 10019, United States', 'Website': 'thomaskeller.com', 'Phone': '+1 212-823-9335'}
{'Name': 'Per Se', 'Address': '10 Columbus Cir, New York, NY 10019, United States', 'Website': 'thomaskeller.com', 'Phone': '+1 212-823-9335', 'Time': ' Opens 4:30 pm'}
[{'Name': 'Per Se', 'Address': '10 Columbus Cir, New York, NY 10019, United States', 'Website': 'thomaskeller.com', 'Phone': '+1 212-823-9335', 'Time': ' Opens 4:30 pm'}]
https://www.google.com/maps/search/Keens+Steakhouse++72+W+36+St+
{'Name': 'Keens Steakhouse', 'Address': '72 W 36 St, New York, NY 10018, United States', 'Website': 'keens.com', 'Phone': '+1 212-947-3636'}
{'Name': 'Keens Steakhouse', 'Address': '72 W 36 St, New York, NY 10018, United States', 'Website': 'keens.com', 'Phone': '+1 212-947-3636', 'Time': ' Opens 11:45 am'}
[{'Name': 'Keens Steakhouse', 'Address': '72 W 36 St, New York, NY 10018, United St

[{'Name': 'Per Se',
  'Address': '10 Columbus Cir, New York, NY 10019, United States',
  'Website': 'thomaskeller.com',
  'Phone': '+1 212-823-9335',
  'Time': ' Opens 4:30 pm'},
 {'Name': 'Keens Steakhouse',
  'Address': '72 W 36 St, New York, NY 10018, United States',
  'Website': 'keens.com',
  'Phone': '+1 212-947-3636',
  'Time': ' Opens 11:45 am'},
 {'Name': 'Marc Forgione',
  'Address': '134 Reade St, New York, NY 10013, United States',
  'Website': 'marcforgione.com',
  'Phone': '+1 212-941-9401',
  'Time': ' Opens 5 pm'},
 {'Name': 'Le Coucou',
  'Address': '138 Lafayette St, New York, NY 10013, United States',
  'Website': 'lecoucou.com',
  'Phone': '+1 212-271-4252',
  'Time': ' Opens 11:30 am'}]

In [191]:
final = x(df)

https://www.google.com/maps/search/Berimbau+Brazilian+Kitchen++43+Carmine+St+
[{'Name': 'Berimbau Brazilian Kitchen', 'Address': '43 Carmine St, New York, NY 10014, United States', 'Website': 'berimbaunyc.com', 'Phone': '+1 212-401-0021', 'Time': ' Opens 11:30 am'}]
https://www.google.com/maps/search/Victory+Sweet+Shop/Victory+Garden+Cafe++21-69+Steinway+St+
[{'Name': 'Victory Sweet Shop/Victory Garden Cafe', 'Address': '21-69 Steinway St, Queens, NY 11105, United States', 'Website': 'victorysweetshop.com', 'Phone': '+1 718-274-2087', 'Time': ' Opens 8 am'}]
https://www.google.com/maps/search/Per+Se++10+Columbus+Cir+
[{'Name': 'Per Se', 'Address': '10 Columbus Cir, New York, NY 10019, United States', 'Website': 'thomaskeller.com', 'Phone': '+1 212-823-9335', 'Time': ' Opens 4:30 pm'}]
https://www.google.com/maps/search/Marc+Forgione++134+Reade+St+
[{'Name': 'Marc Forgione', 'Address': '134 Reade St, New York, NY 10013, United States', 'Website': 'marcforgione.com', 'Phone': '+1 212-941

In [234]:
xsdf = x(df)

https://www.google.com/maps/search/Berimbau+Brazilian+Kitchen++43+Carmine+St
[{'Name': 'Berimbau Brazilian Kitchen', 'Address': '43 Carmine St, New York, NY 10014, United States', 'Email': 'berimbaunyc.com', 'Phone': '+1 212-401-0021'}]
https://www.google.com/maps/search/Victory+Sweet+Shop/Victory+Garden+Cafe++21-69+Steinway+St
[{'Name': 'Victory Sweet Shop/Victory Garden Cafe', 'Address': '21-69 Steinway St, Queens, NY 11105, United States', 'Email': 'victorysweetshop.com', 'Phone': '+1 718-274-2087'}]
https://www.google.com/maps/search/Per+Se++10+Columbus+Cir
[{'Name': 'Per Se', 'Address': '10 Columbus Cir, New York, NY 10019, United States', 'Email': 'thomaskeller.com', 'Phone': '+1 212-823-9335'}]
https://www.google.com/maps/search/Keens+Steakhouse++72+W+36+St
[{'Name': 'Keens Steakhouse', 'Address': '72 W 36 St, New York, NY 10018, United States', 'Email': 'keens.com', 'Phone': '+1 212-947-3636'}]
https://www.google.com/maps/search/Marc+Forgione++134+Reade+St
[{'Name': 'Marc Forgi

In [192]:
df

,Name,Rating,Reviews,Address,Category,Time
0,Berimbau Brazilian Kitchen,4.5,958,43 Carmine St,Brazilian,Opens 11:30 am
1,Victory Sweet Shop/Victory Garden Cafe,4.5,342,21-69 Steinway St,Restaurant,Opens 8 am
2,Per Se,4.5,"1,525",10 Columbus Cir,French,Opens 4:30 pm
3,Marc Forgione,4.6,580,134 Reade St,American,Opens 5 pm
4,Le Coucou,4.5,"1,420",138 Lafayette St,French,Opens 11:30 am
...,...,...,...,...,...,...
108,Yard House,4.5,"2,059",575 7th Ave,New American,Opens 11 AM
109,Au Cheval,4.5,"1,413",33 Cortlandt Alley,American,Opens 12 PM
110,Dos Caminos,4.0,"1,840",1567 Broadway,Mexican,Opens 7 AM
111,Osteria Carlina West Village,4.8,558,455 Hudson St,Northern Italian,Opens 4:30 PM


In [204]:
salman = pd.DataFrame(final)

In [205]:
salman

,Name,Address,Website,Phone,Time
0,Berimbau Brazilian Kitchen,"43 Carmine St, New York, NY 10014, United States",berimbaunyc.com,+1 212-401-0021,Opens 11:30 am
1,Victory Sweet Shop/Victory Garden Cafe,"21-69 Steinway St, Queens, NY 11105, United St...",victorysweetshop.com,+1 718-274-2087,Opens 8 am
2,Per Se,"10 Columbus Cir, New York, NY 10019, United St...",thomaskeller.com,+1 212-823-9335,Opens 4:30 pm
3,Marc Forgione,"134 Reade St, New York, NY 10013, United States",marcforgione.com,+1 212-941-9401,Opens 5 pm
4,Le Coucou,"138 Lafayette St, New York, NY 10013, United S...",lecoucou.com,+1 212-271-4252,Opens 11:30 am
...,...,...,...,...,...
108,Yard House,"575 7th Ave, New York, NY 10018, United States",yardhouse.com,+1 646-434-2448,Opens 11 am
109,Au Cheval,"33 Cortlandt Alley, New York, NY 10013, United...",auchevalnyc.com,+1 646-350-2429,Opens 8 am
110,Dos Caminos,"1567 Broadway, New York, NY 10036, United States",doscaminos.com,+1 212-918-1330,Opens 7 am
111,Osteria Carlina West Village,"455 Hudson St, New York, NY 10014, United States",osteriacarlina.com,+1 646-559-5137,Opens 4:30 pm


In [206]:
salman

,Name,Address,Website,Phone,Time
0,Berimbau Brazilian Kitchen,"43 Carmine St, New York, NY 10014, United States",berimbaunyc.com,+1 212-401-0021,Opens 11:30 am
1,Victory Sweet Shop/Victory Garden Cafe,"21-69 Steinway St, Queens, NY 11105, United St...",victorysweetshop.com,+1 718-274-2087,Opens 8 am
2,Per Se,"10 Columbus Cir, New York, NY 10019, United St...",thomaskeller.com,+1 212-823-9335,Opens 4:30 pm
3,Marc Forgione,"134 Reade St, New York, NY 10013, United States",marcforgione.com,+1 212-941-9401,Opens 5 pm
4,Le Coucou,"138 Lafayette St, New York, NY 10013, United S...",lecoucou.com,+1 212-271-4252,Opens 11:30 am
...,...,...,...,...,...
108,Yard House,"575 7th Ave, New York, NY 10018, United States",yardhouse.com,+1 646-434-2448,Opens 11 am
109,Au Cheval,"33 Cortlandt Alley, New York, NY 10013, United...",auchevalnyc.com,+1 646-350-2429,Opens 8 am
110,Dos Caminos,"1567 Broadway, New York, NY 10036, United States",doscaminos.com,+1 212-918-1330,Opens 7 am
111,Osteria Carlina West Village,"455 Hudson St, New York, NY 10014, United States",osteriacarlina.com,+1 646-559-5137,Opens 4:30 pm


In [207]:
df

,Name,Rating,Reviews,Address,Category,Time
0,Berimbau Brazilian Kitchen,4.5,958,43 Carmine St,Brazilian,Opens 11:30 am
1,Victory Sweet Shop/Victory Garden Cafe,4.5,342,21-69 Steinway St,Restaurant,Opens 8 am
2,Per Se,4.5,"1,525",10 Columbus Cir,French,Opens 4:30 pm
3,Marc Forgione,4.6,580,134 Reade St,American,Opens 5 pm
4,Le Coucou,4.5,"1,420",138 Lafayette St,French,Opens 11:30 am
...,...,...,...,...,...,...
108,Yard House,4.5,"2,059",575 7th Ave,New American,Opens 11 AM
109,Au Cheval,4.5,"1,413",33 Cortlandt Alley,American,Opens 12 PM
110,Dos Caminos,4.0,"1,840",1567 Broadway,Mexican,Opens 7 AM
111,Osteria Carlina West Village,4.8,558,455 Hudson St,Northern Italian,Opens 4:30 PM


In [208]:
q = df.drop("Address", 1)
q

,Name,Rating,Reviews,Category,Time
0,Berimbau Brazilian Kitchen,4.5,958,Brazilian,Opens 11:30 am
1,Victory Sweet Shop/Victory Garden Cafe,4.5,342,Restaurant,Opens 8 am
2,Per Se,4.5,"1,525",French,Opens 4:30 pm
3,Marc Forgione,4.6,580,American,Opens 5 pm
4,Le Coucou,4.5,"1,420",French,Opens 11:30 am
...,...,...,...,...,...
108,Yard House,4.5,"2,059",New American,Opens 11 AM
109,Au Cheval,4.5,"1,413",American,Opens 12 PM
110,Dos Caminos,4.0,"1,840",Mexican,Opens 7 AM
111,Osteria Carlina West Village,4.8,558,Northern Italian,Opens 4:30 PM


In [209]:
q = q.drop("Time", 1)
q

,Name,Rating,Reviews,Category
0,Berimbau Brazilian Kitchen,4.5,958,Brazilian
1,Victory Sweet Shop/Victory Garden Cafe,4.5,342,Restaurant
2,Per Se,4.5,"1,525",French
3,Marc Forgione,4.6,580,American
4,Le Coucou,4.5,"1,420",French
...,...,...,...,...
108,Yard House,4.5,"2,059",New American
109,Au Cheval,4.5,"1,413",American
110,Dos Caminos,4.0,"1,840",Mexican
111,Osteria Carlina West Village,4.8,558,Northern Italian


In [210]:
a = q.merge(salman, on='Name')

In [211]:
a

,Name,Rating,Reviews,Category,Address,Website,Phone,Time
0,Berimbau Brazilian Kitchen,4.5,958,Brazilian,"43 Carmine St, New York, NY 10014, United States",berimbaunyc.com,+1 212-401-0021,Opens 11:30 am
1,Victory Sweet Shop/Victory Garden Cafe,4.5,342,Restaurant,"21-69 Steinway St, Queens, NY 11105, United St...",victorysweetshop.com,+1 718-274-2087,Opens 8 am
2,Per Se,4.5,"1,525",French,"10 Columbus Cir, New York, NY 10019, United St...",thomaskeller.com,+1 212-823-9335,Opens 4:30 pm
3,Marc Forgione,4.6,580,American,"134 Reade St, New York, NY 10013, United States",marcforgione.com,+1 212-941-9401,Opens 5 pm
4,Le Coucou,4.5,"1,420",French,"138 Lafayette St, New York, NY 10013, United S...",lecoucou.com,+1 212-271-4252,Opens 11:30 am
...,...,...,...,...,...,...,...,...
108,Yard House,4.5,"2,059",New American,"575 7th Ave, New York, NY 10018, United States",yardhouse.com,+1 646-434-2448,Opens 11 am
109,Au Cheval,4.5,"1,413",American,"33 Cortlandt Alley, New York, NY 10013, United...",auchevalnyc.com,+1 646-350-2429,Opens 8 am
110,Dos Caminos,4.0,"1,840",Mexican,"1567 Broadway, New York, NY 10036, United States",doscaminos.com,+1 212-918-1330,Opens 7 am
111,Osteria Carlina West Village,4.8,558,Northern Italian,"455 Hudson St, New York, NY 10014, United States",osteriacarlina.com,+1 646-559-5137,Opens 4:30 pm


In [215]:
a[a['Website'].isna() == True]

,Name,Rating,Reviews,Category,Address,Website,Phone,Time
14,Restaurant Above,4.3,37,American,"234 W 42nd St, New York, NY 10036, United States",NaN,NaN,Opens 6:30 am
31,La Rotisserie,4.5,404,Bistro,"30 E 20th St, New York, NY 10003, United States",NaN,+1 212-267-7426,Opens 5:30 pm
77,Kaenoy Restaurant,5.0,1,Japanese,"623 9th Ave, New York, NY 10036, United States",NaN,+1 212-266-6715,NaN
93,Mari Vanna,4.5,"1,403",Eastern European,"41 E 20th St, New York, NY 10003, United States",NaN,+1 212-777-1955,Opens 4 pm
99,Chef Pnina,2.0,1,Fine Dining,"200 Central Park S, New York, NY 10019, United...",NaN,+1 614-669-2433,Opens 12 pm


In [216]:
a.to_csv("list.csv")

In [220]:
dasf = x(df)

https://www.google.com/maps/search/Berimbau+Brazilian+Kitchen++43+Carmine+St
[{'Address': '43 Carmine St, New York, NY 10014, United States', 'Email': 'berimbaunyc.com', 'Phone': '+1 212-401-0021'}]
https://www.google.com/maps/search/Victory+Sweet+Shop/Victory+Garden+Cafe++21-69+Steinway+St
[{'Address': '21-69 Steinway St, Queens, NY 11105, United States', 'Email': 'victorysweetshop.com', 'Phone': '+1 718-274-2087'}]
https://www.google.com/maps/search/Per+Se++10+Columbus+Cir
[{'Address': '10 Columbus Cir, New York, NY 10019, United States', 'Email': 'thomaskeller.com', 'Phone': '+1 212-823-9335'}]
https://www.google.com/maps/search/Keens+Steakhouse++72+W+36+St
[{'Address': '72 W 36 St, New York, NY 10018, United States', 'Email': 'keens.com', 'Phone': '+1 212-947-3636'}]
https://www.google.com/maps/search/Marc+Forgione++134+Reade+St
[{'Address': '134 Reade St, New York, NY 10013, United States', 'Email': 'marcforgione.com', 'Phone': '+1 212-941-9401'}]
https://www.google.com/maps/searc

In [221]:
gilani = pd.DataFrame(dasf)

In [222]:
gilani

,Address,Email,Phone
0,"43 Carmine St, New York, NY 10014, United States",berimbaunyc.com,+1 212-401-0021
1,"21-69 Steinway St, Queens, NY 11105, United St...",victorysweetshop.com,+1 718-274-2087
2,"10 Columbus Cir, New York, NY 10019, United St...",thomaskeller.com,+1 212-823-9335
3,"72 W 36 St, New York, NY 10018, United States",keens.com,+1 212-947-3636
4,"134 Reade St, New York, NY 10013, United States",marcforgione.com,+1 212-941-9401
...,...,...,...
109,"29 W 26th St # 1, New York, NY 10010, United S...",latinamericanrest.com,+1 212-689-2570
110,"62 Thomas St, New York, NY 10013, United States",buddhabarnewyork.com,+1 212-256-0360
111,"135 W Broadway, New York, NY 10013, United States",tinysnyc.com,+1 212-374-1135
112,"33 Cortlandt Alley, New York, NY 10013, United...",auchevalnyc.com,+1 646-350-2429


In [223]:
df.merge(gilani, on='Address')

,Name,Rating,Reviews,Address,Category,Time,Email,Phone


In [158]:
pd.DataFrame(temp)

,0,1,2,3,4,5,6
0,"43 Carmine St, New York, NY 10014, United States",Place an order,berimbaunyc.com,+1 212-401-0021,"PXJW+2H New York, USA",Send to your phone,None
1,"21-69 Steinway St, Queens, NY 11105, United St...",Find a table,Place an order,victorysweetshop.com,+1 718-274-2087,"Q3FV+6R New York, USA",Send to your phone
2,"10 Columbus Cir, New York, NY 10019, United St...",Floor 4 · The Shops at Columbus Circle,thomaskeller.com,+1 212-823-9335,"Q298+7R New York, USA",Send to your phone,None
3,"72 W 36 St, New York, NY 10018, United States",keens.com,+1 212-947-3636,"Q227+8C New York, USA",Send to your phone,None,None
4,"134 Reade St, New York, NY 10013, United States",marcforgione.com,+1 212-941-9401,"PX8R+H5 New York, USA",Send to your phone,None,None


In [149]:
information_extractor("https://www.google.com/maps/search/Berimbau+Brazilian+Kitchen++43+Carmine+St")

43 Carmine St, New York, NY 10014, United States
Place an order
berimbaunyc.com
+1 212-401-0021
PXJW+2H New York, USA
Send to your phone
['43 Carmine St, New York, NY 10014, United States', 'Place an order', 'berimbaunyc.com', '+1 212-401-0021', 'PXJW+2H New York, USA', 'Send to your phone']


['43 Carmine St, New York, NY 10014, United States',
 'Place an order',
 'berimbaunyc.com',
 '+1 212-401-0021',
 'PXJW+2H New York, USA',
 'Send to your phone']

In [129]:
information_extractor(url)

114 Kenmare St, New York, NY 10012, United States
Find a table
esquinanyc.com
+1 646-613-7100
P2C2+HX New York, USA
Send to your phone
Send to your phone


In [22]:
import multiprocessing

# Number of processes to use (you can adjust this based on your system resources)
num_processes = multiprocessing.cpu_count()  # Using all available CPU cores

# Create a multiprocessing pool with the specified number of processes
with multiprocessing.Pool(processes=4) as pool:
    # Map the process_list_item function to the input_list and get the results
    results = pool.map(xtractor, df[0:10])

print("Original list:", df)
print("Processed list:", results)

In [9]:
df = pd.read_csv("new_data.csv")

In [10]:
df.drop("Unnamed: 0", 1, inplace=True)

In [11]:
df

,Name,Rating,Reviews,Address,Category,Time
0,Victory Sweet Shop/Victory Garden Cafe,4.5,342,21-69 Steinway St,Restaurant,Opens 8 am
1,Berimbau Brazilian Kitchen,4.5,958,43 Carmine St,Brazilian,Opens 11:30 am
2,Per Se,4.5,"1,525",10 Columbus Cir,French,Opens 4:30 pm
3,Keens Steakhouse,4.6,"7,028",72 W 36 St,Steak,Opens 11:45 am
4,Marc Forgione,4.6,580,134 Reade St,American,Opens 5 pm
...,...,...,...,...,...,...
108,Nocello,4.6,304,257 W 55th St,Italian,Opens 4 PM
109,Latin American,4.2,237,29 W 26th St # 1,Dominican,Opens 7 AM
110,Buddha-Bar Restaurant New York,4.0,481,62 Thomas St,Asian Fusion,NaN
111,Tiny's & The Bar Upstairs,4.4,822,135 W Broadway,American,Opens 5 PM


In [12]:
import multiprocessing

 

def process_list_item(item):
    # Your processing logic for each item goes here
    # For demonstration purposes, let's just square the item
    return item ** 2

 

if __name__ == "__main__":
    # Sample list
    input_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

 

    # Number of processes to use (you can adjust this based on your system resources)
    num_processes = multiprocessing.cpu_count()  # Using all available CPU cores

 

    # Create a multiprocessing pool with the specified number of processes
    with multiprocessing.Pool(processes=num_processes) as pool:
        # Map the process_list_item function to the input_list and get the results
        results = pool.map(process_list_item, input_list)

 

    print("Original list:", input_list)
    print("Processed list:", results)